# Query Google Earth Engine (GEE) for reference DEM for the `skysat_triplet_pipeline`

In [2]:
import os
import glob
import ee
import sys

## Define input and output files

In [ ]:
buffer = 30e3 # [m] how much to buffer the AOI before clipping DEM 

site_name = 'Variegated'
im_path = f'/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites/{site_name}/20240419/SkySatScene/'
dem_path = f'/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites/{site_name}/refdem/'
code_path = '/Users/raineyaberle/Research/PhD/SnowDEMs/skysat-snow'

# Import utility functions
sys.path.append(os.path.join(code_path, 'scripts'))
import query_refdem_utils as f


## Calculate image bounds and buffer

In [ ]:
# Load metadata file names
meta_fns = sorted(glob.glob(os.path.join(im_path, '*_metadata.json')))

# Create clipping region
bbox, epsg_utm = f.create_bbox_from_meta(meta_fns, buffer)


## Initialize GEE

In [7]:
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Intialize()

## Query GEE for DEM, clip, save to file

In [ ]:
# Make sure output directory exists
if not os.path.exists(dem_path):
    os.mkdir(dem_path)
    print('Made directory for output file:', dem_path)
# Define output file name
dem_fn = os.path.join(dem_path, f"{site_name}_ArcticDEM_buffer{int(buffer/1e3)}km.tif")
# Query and download DEM
dem_ds = f.query_gee_for_arcticdem(bbox, out_fn=dem_fn, crs=epsg_utm)